In [201]:
import Pkg; Pkg.add("MultiObjectiveAlgorithms")
import DataFrames
Pkg.add("Ipopt")
import MultiObjectiveAlgorithms as MOA
Pkg.add("Plots")
Pkg.add("Statistics")
Pkg.add("StatsPlots")

import Pkg;
Pkg.add("YFinance")
Pkg.add("Statistics")


   Resolving package versions...


  No Changes to `C:\Users\Nicol\.julia\environments\v1.9\Project.toml`
  No Changes to `C:\Users\Nicol\.julia\environments\v1.9\Manifest.toml`


   Resolving package versions...


  No Changes to `C:\Users\Nicol\.julia\environments\v1.9\Project.toml`
  No Changes to `C:\Users\Nicol\.julia\environments\v1.9\Manifest.toml`

   Resolving package versions...

...


  No Changes to `C:\Users\Nicol\.julia\environments\v1.9\Project.toml`
  No Changes to `C:\Users\Nicol\.julia\environments\v1.9\Manifest.toml`


   Resolving package versions...


  No Changes to `C:\Users\Nicol\.julia\environments\v1.9\Project.toml`
  No Changes to `C:\Users\Nicol\.julia\environments\v1.9\Manifest.toml`

   Resolving package versions...


  No Changes to `C:\Users\Nicol\.julia\environments\v1.9\Project.toml`
  No Changes to `C:\Users\Nicol\.julia\environments\v1.9\Manifest.toml`


In [205]:
statu

ErrorException: syntax: extra token "DataFrames" after end of expression

In [197]:
using YFinance
data=get_prices("NFLX",interval="1mo",startdt=DateTime(2015,10,1),enddt=DateTime(2020,10,30))


OrderedCollections.OrderedDict{String, Any} with 8 entries:
  "ticker"    => "NFLX"
  "timestamp" => [DateTime("2015-10-01T04:00:00"), DateTime("2015-11-01T04:00:0…
  "open"      => [102.91, 109.2, 124.47, 109.0, 91.79, 94.58, 102.93, 90.41, 10…
  "high"      => [115.83, 126.6, 133.27, 122.18, 97.48, 104.91, 111.85, 104.0, …
  "low"       => [96.26, 101.86, 113.85, 90.11, 79.95, 93.61, 88.21, 85.74, 84.…
  "close"     => [108.38, 123.33, 114.38, 91.84, 93.41, 102.23, 90.03, 102.57, …
  "adjclose"  => [108.38, 123.33, 114.38, 91.84, 93.41, 102.23, 90.03, 102.57, …
  "vol"       => [4.46204e8, 3.20322e8, 3.19939e8, 4.88193e8, 3.89269e8, 3.1133…

In [198]:
using YFinance
using Statistics
using LinearAlgebra

PF=["NFLX","IBM"]
data=get_prices.(PF,interval="1mo",startdt=DateTime(2015,10,1),enddt=DateTime(2020,10,30)) # récupere les data sur une plage de temps


k=1
for j in PF                     # boucle pour chaque action de la liste
    j=data[k]["open"]
    for i = 1:n-1           # colonne des variation entre 2 itérations
        
        diff=(j[i+1]-j[i])/j[i] # évolution entre 2 itérations en %

        if i==1 
            E=diff
        else 
            E=[E diff]
        end
    end
    println(E)

    if k==1         # concaténation des 2 colonnes pour faieune matrice
        M=E'
    else 
        cop=M

        M=hcat(cop,E')
        println(M)
    end
    k=k+1
end

# M est une matrice regroupant en colonne l'évolution mensuelle en %
m=vec(Statistics.mean(M; dims = 1))
Q = Statistics.cov(M)  # Matrice de covariance

[0.061121300770575496 0.13983520786816964 -0.12428698536985308 -0.1578898998575473 0.03039547758687714 0.08828503185098723 -0.1216360303696302 0.12266337671368127 -0.06403940886699508 -0.0396841751901727 0.0721253311360321 0.0019425666715964054]
[-0.033101762027059445 0.002770823421398497 -0.028514372593783137 -0.08259562074162323 0.0738809292942458 0.13815752425138672 -0.026244095212505925 0.054015721399928086 -0.007973836788128456 0.058439766479962886 -0.0059923413246444875 -0.0016420710729600704]
[0.061121300770575496 -0.033101762027059445; 0.13983520786816964 0.002770823421398497; -0.12428698536985308 -0.028514372593783137; -0.1578898998575473 -0.08259562074162323; 0.03039547758687714 0.0738809292942458; 0.08828503185098723 0.13815752425138672; -0.1216360303696302 -0.026244095212505925; 0.12266337671368127 0.054015721399928086; -0.06403940886699508 -0.007973836788128456; -0.0396841751901727 0.058439766479962886; 0.0721253311360321 -0.0059923413246444875; 0.0019425666715964054 -0.00

2×2 Matrix{Float64}:
 0.0102819   0.00335112
 0.00335112  0.00352243

Maintnant que l'on a la matrice de covariance nous allons chercher à minimiser x'Q x (qui est la variance)
On se fixe une rentabilité à viser, imaginons 4%, soit pour un montant initial de 10 000, 400$ donc  r'*x>=400 et somme(x)<10 000

In [203]:
using JuMP
import DataFrames
import Ipopt
import MultiObjectiveAlgorithms as MOA
import Plots
import Statistics
import StatsPlots

model = Model(Ipopt.Optimizer)

@variable(model, x[1:3] >= 0)

@objective(model, Min, x' * Q * x)

@constraint(model, sum(x) <= 1000)
@constraint(model, r' * x >= 50)

optimize!(model)
solution_summary(model)

ERROR: LoadError: InitError: 

could not load library "C:\Users\Nicol\.julia\artifacts\f133cc1941896efc3f746ac449821136a1a0c750\bin\libopenblas.dll"
The specified module could not be found. 


Stacktrace:
  [1] dlopen(s::String, flags::UInt32; throw_error::Bool)
    @ Base.Libc.Libdl .\libdl.jl:117
  [2] dlopen(s::String, flags::UInt32)
    @ Base.Libc.Libdl .\libdl.jl:116
  [3] macro expansion
    @ C:\Users\Nicol\.julia\packages\JLLWrappers\pG9bm\src\products\library_generators.jl:63 [inlined]
  [4] __init__()
    @ OpenBLAS32_jll C:\Users\Nicol\.julia\packages\OpenBLAS32_jll\0chkR\src\wrappers\x86_64-w64-mingw32-libgfortran5.jl:9
  [5] register_restored_modules(sv::Core.SimpleVector, pkg::Base.PkgId, path::String)
    @ Base .\loading.jl:1115
  [6] _include_from_serialized(pkg::Base.PkgId, path::String, ocachepath::String, depmods::Vector{Any})
    @ Base .\loading.jl:1061
  [7] _require_search_from_serialized(pkg::Base.PkgId, sourcepath::String, build_id::UInt128)
    @ Base .\loading.jl:1506
  [8] _require(pkg::Base.PkgId, env::String)
    @ Base .\loading.jl:1783
  [9] _require_prelocked(uuidkey::Base.PkgId, env::String)
    @ Base .\loading.jl:1660
 [10] macro expans

ErrorException: Failed to precompile Ipopt [b6b21f68-93f8-5de0-b562-5493be1d77c9] to "C:\\Users\\Nicol\\.julia\\compiled\\v1.9\\Ipopt\\jl_2911.tmp".